# Notebook to explore netCDF files and change resolution, plus Python plotting
These files are downloaded from [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=download), using `cdsapi`. Get the data running the Python script `inout.py`:
```
python onehealth_db/inout.py
```

The downloaded files are stored in `data/in`. The `area` option uses values `90`, `90`, `-90`, `-90` for `North`, `East`, `South`, `West`, respectively.

Question: What is the coordinate reference system for the era5 dataset? NUTS3 either on EPSG 3035, 4326, 3857.

-> According to [ERA5-Land's documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation):
> The data is referenced in the horizontal with respect to the WGS84 ellipse (which defines the major/minor axes) and in the vertical it is referenced to the EGM96 geoid over land but over ocean it is referenced to mean sea level, with the approximation that this is assumed to be coincident with the geoid. 

Then according to [this page](https://spatialreference.org/ref/epsg/9707/), it seems like the coordinate reference system for ERA5-Land is EPSG:9707

> ERA5-Land produces a total of 50 variables describing the
water and energy cycles over land, globally, hourly, and at a
spatial resolution of 9 km, matching the ECMWF triangular–
cubic–octahedral (TCo1279) operational grid (Malardel
et al., 2016).

In [ ]:
from pathlib import Path
import xarray as xr
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

The following cells aim to explore the data structure

In [ ]:
data_folder = Path("../../../data/")

### ERA5-Land from CDS

In [ ]:
f_area_before_celsius = (
    data_folder / "in" / "era5_data_2016_2017_all_2t_tp_monthly_raw.nc"
)
f_area_after_celsius = (
    data_folder
    / "processed"
    / "era5_data_2016_2017_all_2t_tp_monthly_unicoords_adjlon_celsius_mm_05deg_trim_ts20250827-102946_hssc08.nc"
)

#### Dask Array

In [ ]:
dask_ds = xr.open_dataset(f_area_after_celsius, chunks={})
dask_ds = dask_ds.chunk({"time": 1, "latitude": 900, "longitude": 1800})
dask_ds

In [ ]:
t2m_data = (
    dask_ds["t2m"].dropna(dim="latitude", how="all").load()
)  # load data into memory
t2m_data

In [ ]:
stacked = t2m_data.stack(points=("time", "latitude", "longitude"))
stacked

In [ ]:
stacked = stacked.dropna("points")
stacked

In [ ]:
stacked["time"].values.astype("datetime64[ns]")

In [ ]:
stacked["latitude"].values

In [ ]:
stacked["longitude"].values

In [ ]:
stacked.values

#### Xarray

In [ ]:
# load netCDF files
ds_area_before_celsius = xr.open_dataset(f_area_before_celsius)
ds_area_after_celsius = xr.open_dataset(f_area_after_celsius)

In [ ]:
ds_area_before_celsius

In [ ]:
ds_area_before_celsius.sel(
    latitude=20.0, longitude=10.0, method="nearest"
).to_dataframe().head(5)

In [ ]:
ds_area_before_celsius["tp"].attrs

In [ ]:
ds_area_after_celsius

In [ ]:
ds_area_after_celsius.latitude.values[5]

In [ ]:
ds_area_after_celsius["tp"].attrs

In [ ]:
ds_area_after_celsius.sel(
    latitude=20.0, longitude=10.0, method="nearest"
).to_dataframe().head(5)

In [ ]:
ds_area_after_celsius.latitude.values[5]

In [ ]:
lat = 20.0
lon = 10.0
ds_area_after_celsius["t2m"].sel(latitude=lat, longitude=lon, method="nearest").plot(
    color="blue", marker="o"
)
plt.title("2m temperature in 2024 at lat-{}, lon-{}".format(lat, lon))
plt.show()

In [ ]:
# plot the data for the first month
ds_area_after_celsius.t2m[0].plot(size=7)

In [ ]:
ds_area_after_celsius.tp[0].plot(size=7)

In [ ]:
# convert to dataframe
df = ds_area_after_celsius.to_dataframe().reset_index()
df

### Population data from ISIMIP

In [ ]:
f_popu_data = (
    data_folder
    / "processed"
    / "population_histsoc_30arcmin_annual_1901_2021_unicoords_2016_2021_ts20250827-102950_hssc08.nc"
)
ds_popu_data = xr.open_dataset(f_popu_data, chunks={})

In [ ]:
ds_popu_data

In [ ]:
# only keep data from 2016 and 2017
end_time = np.datetime64("2017-12-01", "ns")
limit_time = np.datetime64("2016-01-01", "ns")
ds_popu_data = ds_popu_data.sel(time=slice(limit_time, end_time))
ds_popu_data.time.values

In [ ]:
if ds_popu_data.time.values[0] > ds_popu_data.time.values[-1]:
    # sort the time dimension in ascending order
    ds_popu_data = ds_popu_data.sortby("time")
start_of_year = pd.Timestamp(
    year=pd.to_datetime(ds_popu_data.time.values[0]).year,
    month=1,
    day=1,
    hour=12,
    minute=0,  # 0 hours for era5 data
)
end_of_year = pd.Timestamp(
    year=pd.to_datetime(ds_popu_data.time.values[-1]).year,
    month=12,
    day=1,
    hour=12,
    minute=0,
)
monthly_time = pd.date_range(start=start_of_year, end=end_of_year, freq="MS")
monthly_time

In [ ]:
# reindex the time dimension to match the monthly time
ds_popu_data = ds_popu_data.reindex(time=monthly_time, method="ffill")
ds_popu_data

In [ ]:
ds_popu_data["total-population"].sel(
    latitude=8.67, longitude=49.39, method="nearest"
).to_dataframe().head(14)

In [ ]:
ds_popu_data["total-population"].attrs

In [ ]:
# resolution of population data
res = ds_popu_data.latitude[1] - ds_popu_data.latitude[0]
res

In [ ]:
test_popu_data = ds_popu_data.sel(
    latitude=8.67, longitude=49.39, method="nearest"
).to_dataframe()
test_popu_data.head(5)

In [ ]:
test_popu_data["total-population"].plot()

In [ ]:
ds_popu_data["total-population"][-1].plot(figsize=(9, 5))

## Check all the data visually
### Cartesian grids

In [ ]:
cartesian_grid_file = (
    data_folder
    / "processed"
    / "era5_data_2016_2017_all_2t_tp_monthly_unicoords_adjlon_celsius_mm_05deg_trim_ts20250827-102946_hssc08.nc"
)
pop_cartesian_grid_file = (
    data_folder
    / "processed"
    / "population_histsoc_30arcmin_annual_1901_2021_unicoords_2016_2021_ts20250827-102950_hssc08.nc"
)

In [ ]:
# read the netcdf data into xarray
cartesian_grid = xr.open_dataset(cartesian_grid_file)
pop_cartesian_grid = xr.open_dataset(pop_cartesian_grid_file)

In [ ]:
# plot the cartesian grid data of t2m and tp for 2016-2017, all months
cartesian_grid.t2m.plot(col="time", col_wrap=4, cmap="coolwarm", figsize=(15, 10))
plt.savefig("era5_2016_2017_plots_t2m.png", dpi=300)
plt.show()

In [ ]:
cartesian_grid.tp.plot(
    col="time", col_wrap=4, cmap="coolwarm", vmin=0, vmax=20, figsize=(15, 10)
)
plt.savefig("era5_2016_2017_plots_tp.png", dpi=300)
plt.show()

In [ ]:
# plot the cartesian population data
pop_cartesian_grid["total-population"].plot(
    col="time", col_wrap=2, cmap="coolwarm", vmin=0, vmax=0.8e6, figsize=(15, 10)
)
plt.savefig("population_2016_2017_plots.png", dpi=300)
plt.show()

### NUTS averaged data

In [ ]:
nuts_aggregated_file = (
    data_folder / "processed" / "NUTS_RG_20M_2024_4326_agg_era5_popu_2016-01-2021-01.nc"
)

In [ ]:
# read the netcdf data into xarray
nuts_grid = xr.open_dataset(nuts_aggregated_file)

In [ ]:
nuts_grid

In [ ]:
# convert the NUTS3 shape string into a shapely geometry
# NUTS3_grid["geometry"] = NUTS3_grid["geometry"].apply(wkt.loads)
# convert pandas DataFrame to GeoDataFrame
# NUTS3_grid_gdf = gpd.GeoDataFrame(NUTS3_grid, geometry="geometry")
# NUTS3_grid_gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
# era5_merge = gpd.tools.sjoin(NUTS3_grid, nuts3, how="left")

# drop non-merged obs
# era5_matched = era5_merge[~era5_merge["NUTS_NAME"].isna()]
# show result
# era5_matched.head()

# ear5_aggregated_by_NUTS3 = (
#     era5_matched.groupby("NUTS_ID")[["t2m", "tp"]].mean().reset_index()
# )
# ear5_aggregated_by_NUTS3
#
# era5_nuts = nuts3.merge(ear5_aggregated_by_NUTS3, on="NUTS_ID")
# era5_nuts = era5_nuts.filter(["NUTS_ID", "geometry", "t2m", "tp"])
# era5_nuts
#
# fig, ax = plt.subplots(figsize=(8, 5))
# era5_nuts.plot(ax=ax, column="t2m", legend=True, markersize=0.5, cmap="coolwarm")
# plt.tight_layout()
# fig.savefig("nuts3_t2m.png", dpi=300)